In [1]:
# imports
from itertools import permutations 
import numpy as np
import sys
import math
import time
from solverFunctions import *

In [2]:
# test grids
emptySudoku = np.zeros((9,9))

hardSudoku = np.zeros((9,9))
hardSudoku[0,1] = 4
hardSudoku[0,7] = 1
hardSudoku[1,0] = 2
hardSudoku[1,8] = 6
hardSudoku[7,0] = 9
hardSudoku[7,8] = 2
hardSudoku[8,1] = 1
hardSudoku[8,7] = 9

sampleSudoku = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
                         [6, 0, 0, 1, 9, 5, 0, 0, 0],
                         [0, 9, 8, 0, 0, 0, 0, 6, 0],
                         [8, 0, 0, 0, 6, 0, 0, 0, 3],
                         [4, 0, 0, 8, 0, 3, 0, 0, 1],
                         [7, 0, 0, 0, 2, 0, 0, 0, 6],
                         [0, 6, 0, 0, 0, 0, 2, 8, 0],

                         [0, 0, 0, 4, 1, 9, 0, 0, 5],
                         [0, 0, 0, 0, 8, 0, 0, 7, 9]])

In [3]:
def getNArrangFast(grid, selSquare):
    squaresIdx = ([[0, 1, 2], [0, 1, 2]],
                  [[0, 1, 2], [3, 4, 5]],
                  [[0, 1, 2], [6, 7, 8]],
                  [[3, 4, 5], [0, 1, 2]],
                  [[3, 4, 5], [3, 4, 5]],
                  [[3, 4, 5], [6, 7, 8]],
                  [[6, 7, 8], [0, 1, 2]],
                  [[6, 7, 8], [3, 4, 5]],
                  [[6, 7, 8], [6, 7, 8]]
             )

    # Get all permutations 1-9
    perm = list(permutations(list(range(1,10))))
    
    squareIdRow = getSquareID(squaresIdx[selSquare][0][0])
    squareIdCol = getSquareID(squaresIdx[selSquare][1][0])
    square = grid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3]
    line = np.ndarray.flatten(square)
    freeSpots =  len([i for i in line if i == 0])      
                
    if (freeSpots == 0):
        return([])
    
    
    allProbs = []
    allFrees = []
    allPoss = []
    for numProb in range(1,10):
        probSquare = np.zeros((3,3))
        intIdxR = 0
        for rIdx in squaresIdx[selSquare][0]:
            intIdxC = 0;
            for cIdx in squaresIdx[selSquare][1]:              
                if probabilityFunction(grid, rIdx, cIdx, numProb):
                    probSquare[intIdxR, intIdxC] = 1
                intIdxC = intIdxC + 1
            intIdxR = intIdxR + 1      
        line = np.ndarray.flatten(probSquare)
        possibilities =  len([i for i in line if i > 0])
        allPoss.append(possibilities)
        allProbs.append(probSquare)
        allFrees.append(freeSpots)
    
    #get the first minimum values
    minVal = min(i for i in allPoss if i > 0)
    minPos = allPoss.index(minVal)
    triedNumb = minPos + 1
    probGrid = allProbs[minPos]
    probGrid[probGrid == 1] = triedNumb

    keepPerm = []
    line = np.ndarray.flatten(square)
    for i in perm:
        #use only permutations where existing numbers are where they are in the original square 
        diff = np.asarray(line)-np.asarray(i)
        inters = np.where(diff == 0)
        #how many numbers we have in origna grid
        fixedPos = 9 - freeSpots
        if len(inters[0]) == fixedPos: # only intersect the numbers we already have. All of them
            #keep only permutaitons where our number is in the desired positions
            diff = np.asarray(np.ndarray.flatten(probGrid))-np.asarray(i)
            inters = np.where(diff == 0)
            
            if len(inters[0]) == 1:
                keepPerm.append(i)
            
#                 #we still need to eliminate permutations were a number ends up in
#                 #a row or columkn forbidden by the remaining
#                 #of the grid. Easiset way is to put the permutation in the main grid and check with
#                 #validateGrid()
#                 # this is slow. might be faster to remove it entirely
#                 tempGrid = grid.copy()
#                 tempGrid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3] = np.asarray(i).reshape(3,3)
#                 if validateGrid(tempGrid):
#                     keepPerm.append(i)

    return(keepPerm)

In [4]:
b = hardSudoku.copy()

keepPerm1 = getNArrangFast(hardSudoku, 0)
for i1 in keepPerm1:
    bC1 = b.copy()
    bC1[0:3, 0:3] = np.asarray(i1).reshape(3,3)
    res = solveSudoku(bC1) 
    if len(res) == 9:
        break
    keepPerm2 = getNArrangFast(bC1, 1)
    for i2 in keepPerm2:
        bC2 = bC1.copy()
        bC2[0:3, 3:6] = np.asarray(i2).reshape(3,3)
        res = solveSudoku(bC2) 
        if len(res) == 9:
            break
        keepPerm3 = getNArrangFast(bC2, 2)
        for i3 in keepPerm3:
            bC3 = bC2.copy()
            bC3[0:3, 6:9] = np.asarray(i3).reshape(3,3)
            res = solveSudoku(bC3) 
            if len(res) == 9:
                break
            keepPerm4 = getNArrangFast(bC3, 3)
            for i4 in keepPerm4:
                bC4 = bC3.copy()
                bC4[3:6, 0:3] = np.asarray(i4).reshape(3,3)
                res = solveSudoku(bC4) 
                if len(res) == 9:
                    break
                keepPerm5 = getNArrangFast(bC4, 4)
                for i5 in keepPerm5:
                    bC5 = bC4.copy()
                    bC5[3:6, 3:6] = np.asarray(i5).reshape(3,3)
                    res = solveSudoku(bC5) 
                    if len(res) == 9:
                        break
                    keepPerm6 = getNArrangFast(bC5, 5)
                    for i6 in keepPerm6:
                        bC6 = bC5.copy()
                        bC6[3:6, 6:9] = np.asarray(i6).reshape(3,3)
                        res = solveSudoku(bC6) 
                        if len(res) == 9:
                            break
                        keepPerm7 = getNArrangFast(bC6, 6)
                        for i7 in keepPerm7:
                            bC7 = bC6.copy()
                            bC7[6:9, 0:3] = np.asarray(i7).reshape(3,3)
                            res = solveSudoku(bC7) 
                            if len(res) == 9:
                                break
                            keepPerm8 = getNArrangFast(bC7, 7)
                            for i8 in keepPerm8:
                                bC8 = bC7.copy()
                                bC8[6:9, 3:6] = np.asarray(i8).reshape(3,3)
                                res = solveSudoku(bC8) 
                                if len(res) == 9:
                                    break
                                keepPerm9 = getNArrangFast(bC8, 8)
                                for i9 in keepPerm9:
                                    bC9 = bC8.copy()
                                    bC9[6:9, 6:9] = np.asarray(i9).reshape(3,3)
                                    res = solveSudoku(bC9) 
                                    if len(res) == 9:
                                        break
                        
                
        
    
print(res)

KeyboardInterrupt: 